<a href="https://colab.research.google.com/github/NUG30/homework-2-quang-nguyen-dn/blob/main/Assignment2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework 2: Naive Bayes Email Spam Filter

In this homework, we will learn how to implement the Naive Bayes classifier in order to create a simple email spam filter. This spam filter will be trained by test_emails, which will be given by a vector of tuples (emails, spam/nospam). For each tuple the first entry is a string ("email"), and the second entry is 0 or 1, depending whether the email contains spam words.

In [ ]:
dictionary = ['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'today?', 'do', 'your', 'homework', 'want', 'free', 'solutions?', 'hey', 'always', 'ask', 'questions', 'if','have', 'any', 'math', 'not', 'good', 'submit', 'pay'] 

test_emails=[
             ["hello students, please learn for the exam", 0],
             ["hello students, please buy drugs", 1],    
             ["hello, today the sun is shining in nagoya", 0],
             ["lets sell drugs in nagoya", 1],
             ["today learn drugs", 1],
             ["how are you today?", 0],
             ["hello students, please do your homework", 0],
             ["hello, do you want free homework solutions?", 1],
             ["hey students, please always ask questions if you have any", 0],
             ["math is not good", 1],
             ["math is good", 0],
             ["please submit your homework", 0],
             ["please buy questions", 1],
             ["please pay for the exam", 1]          
             ]

d = len(dictionary)
n = len(test_emails)

The feature space for our spam filter will be $\mathcal{X}=\{0,1\}^d$, where $d$ denotes the number of words in the dictionary. For a feature (email) $x \in \mathcal{X}$ the entry $x_i$ for $i=1,\dots,d$ is $1$ if the $i$-th word of the dictionary is contained in the email and $0$ otherwise.  

# **Exercise 1**
Implement a function which creates a feature vector out of an email and a function which creates a training set out of test emails. 

You would need to figure out whether a sentence contains a word, and there are functions in Python that could determine whether a string contains another string. You can consult documentation (and Google) to find out.

In [ ]:
import numpy as np
def email_to_feature(ldict, email):
  x = np.zeros(d, int)       # numpy array VS list
  for i in range(d):
    if ldict[i] in email: x[i] = 1
  return x

def emails_to_test_samples(ldict, test_emails):
  X = np.empty((n, d), int)
  Y = np.empty(n, int)
  for i in range(n):
    X[i] = email_to_feature(ldict, test_emails[i][0])
    Y[i] = test_emails[i][1]
  return X, Y

 **Recall from Lecture 6:**

Given a training set  $\mathcal{T} = \left( (x^{(1)}, y^{(1)}) , \dots, (x^{(n)}, y^{(n)})   \right)$ we calculate for $i=1,\dots,d$ the following numbers
\begin{align*}
\phi_{i\mid y=1} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=1 ) }{2+\sum_{j=1}^n I(y^{(j)}=1)}\,,\\
	\phi_{i\mid y=0} &= \frac{1+\sum_{j=1}^n I(x^{(j)}_i = 1  \,\wedge \, y^{(j)}=0 )}{2+\sum_{j=1}^n I(y^{(j)}=0)}\,,\\
		\phi_{y=1} &= \frac{1+\sum_{j=1}^n I(y^{(j)} = 1)}{2+n} \,.
\end{align*}
Here $I$ denoted the indicator function. We used the laplace smoothing (thats why we have the $1+$ and $2+$) in order to make sure that we will not assume probability $0$ for unknown words.

Now assume we get a new feature (i.e. someone sends us an email) $x \in \{0,1\}^d$. Then we can calculate for each word $i=1,\dots,d$ the probabilities
\begin{align*}
P(x_i = 1 \mid y=1) &= \phi_{i\mid y=1}\,,\qquad &&P(x_i = 1 \mid y=0) = \phi_{i\mid y=0} \,,\\
P(x_i = 0 \mid y=1) &= 1- \phi_{i\mid y=1}\,,\qquad &&P(x_i = 0 \mid y=0) = 1-\phi_{i\mid y=0} \,. \\
\end{align*}

By the Naive Bayes assumption we have for $x \in \{0,1\}^d$
\begin{align*}
P(x \mid y)  = \prod_{i=1}^d P(x_j \mid y)\,.
\end{align*}
The probability of the new email being spam is then
\begin{align*}
P(y=1 \mid x) &= \frac{ P(x \mid y=1) P(y=1)}{P(x)}\\
&= \frac{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1}}{\prod_{i=1}^d P(x_i \mid y = 1) \cdot \phi_{y=1} + \prod_{i=1}^d P(x_i \mid y = 0) (1-\phi_{y=1})} := \frac{A}{A+B}\,.
\end{align*}


# **Exercise 2:** 
Use the above explanation of the Naive Bayes Spam filter and implement a function which gives the probability of an email being spam given the trainings email above. 


In [ ]:
# You can implement the above algorithm in any way you want.
# One possible way:
# - Calculate the phis here
# - Write functions for the probability P(x|y) depending on these phis
# - Use this function in the function spam_percentage 

def get_phi(X, Y):
  result = np.empty((d, 2))

  for i in range(d):
    num0 = 1; num1 = 1; den0 = 2; den1 = 2
    for j in range(n):
      if X[j][i] == 1 and Y[j] == 1: num1 += 1
      elif X[j][i] == 1 and Y[j] == 0: num0 += 1
      if Y[j] == 1: den1 += 1
      elif Y[j] == 0: den0 += 1
    result[i] = [num0/den0, num1/den1]
  return result

def get_phi_y1(Y):
  result = 1
  for y in Y:
    if y == 1: result += 1
  return result / (2+n)

X, Y = emails_to_test_samples(dictionary, test_emails)
phi = get_phi(X, Y)
phi_y1 = get_phi_y1(Y)
#print(phi)
#print(phi_y1)

def P(x, y):
  result = 1
  for i in range(d):
    if x[i] == 1:     result *= phi[i][y]
    elif x[i] == 0:   result *= 1 - phi[i][y]
  return result
  

def spam_percentage(email):
  x = email_to_feature(dictionary, email)
  A = phi_y1 * P(x, 1)
  B = (1 - phi_y1) * P(x, 0)
  return A/(A+B)


Test your spam filter with the following email.


In [ ]:
email="the sun is shining. buy drugs now"
print(spam_percentage(email))
print(spam_percentage("hello students, how are you today? lets buy drugs"))
print(spam_percentage(""))

0.9299429164504411
0.5423728813559321
0.892430278884462



# **Exercise 3**
Extend your spamfilter by creating a dynamical dictionary. Instead of starting with a fixed dictionary, you should now create a dictionary out of a list of emails. 

Write a function `create_dictionary(emails)` which returns a dictionary created from a list of emails (Give as an array of arrays `[text, spam\nospam]`). Make sure that you do not include words more than once into the dictionary.
To implement this function you should look up the function `split()` for a string in Python. To take care of the symbols "." and "," you can use the `replace()` function of a string.

In [ ]:
import re
def create_dictionary(emails):
  #ldict = np.empty(1, 'U')
  ldict = []
  for email, spam_indicator in emails:
    words = re.findall(r'\w+', email)
    #print(words)
    for word in words:
      if not word in ldict:
        list.append(ldict, word)
  return ldict

print(create_dictionary(test_emails))

['hello', 'students', 'please', 'learn', 'for', 'the', 'exam', 'buy', 'drugs', 'today', 'sun', 'is', 'shining', 'in', 'nagoya', 'lets', 'sell', 'how', 'are', 'you', 'do', 'your', 'homework', 'want', 'free', 'solutions', 'hey', 'always', 'ask', 'questions', 'if', 'have', 'any', 'math', 'not', 'good', 'submit', 'pay']
